# Data setup

In [ ]:
from tsai.all import *

In [ ]:
# Paths
root = '' # change as needed
data_root = root + 'data/'
synthetic_root = root + 'results/synthetic_data/'
save_path = root + 'results/models/'
!mkdir -p {save_path}

In [ ]:
# Parameters
g = 2.3
t_use = 50
n_samples = 40000
seed = 0

In [ ]:
# Read data
X = np.load(f'{synthetic_root}G{g}_ts{t_use}/seed{seed}_{int(n_samples/1000)}k_X.npy')
y = np.load(f'{synthetic_root}G{g}_ts{t_use}/seed{seed}_{int(n_samples/1000)}k_y.npy')

In [ ]:
# Make splits
valid_ptc = 0.2
train_ids = list(np.arange(0, int(y.shape[0]*(1.0-valid_ptc))))
valid_ids = list(np.arange(int(y.shape[0]*(1.0-valid_ptc)), y.shape[0]))
splits = (L(train_ids), L(valid_ids))

In [ ]:
check_data(X, y, splits)

In [ ]:
# Set seed
set_seed(18, True)

# Select transforms
tfms  = [None, [TSRegression()]]
batch_tfms = [TSMagMulNoise(magnitude=1.0), TSMagScale(magnitude=0.25)]

# Create dataloaders
dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=16)
dls.rng.seed(18)

In [ ]:
# Plot a batch
dls.show_batch(sharey=True)

In [ ]:
# Metric
a_range = (-1,1)
def rmse_a(inp, targ):
  return rmse(inp, targ)*100/(a_range[1]-a_range[0])

In [ ]:
from fastai.callback.tracker import SaveModelCallback

# Create learner
model = TCN(dls.vars, X.shape[1], layers=4*[80], conv_dropout=0.0, fc_dropout=0.1)
cbs = [SaveModelCallback(monitor='rmse_a', comp=np.less), ShowGraph()]
learn = Learner(dls, model, metrics=[rmse_a], cbs=cbs)

# Training

In [ ]:
# Find LR
learn.lr_find()

In [ ]:
# Train model
learn.fit_one_cycle(30, 3e-3)

In [ ]:
# Save model
learn.export(f'{save_path}/T{t_use}_S{n_samples}.pkl')
learn.save(f'{save_path}/T{t_use}_S{n_samples}')